### Building models

In [ ]:
from pymgpipe import *

build_models(
    coverage_file='normCoverage.csv',
    taxa_dir='taxaModels',
    out_dir='out',
    diet_fecal_compartments=True,
    coupling_constraints=True,
    compress=True,
    threads=-1,
    parallel=True,
    solver='gurobi',
    write_lp=True,
    cobra_type='.xml',
    lp_type='.mps',
    diet='personalized_diet.csv'
)

### Loading models

In [ ]:
from pymgpipe import *

# Loading a cobra model
cobra_model = load_cobra_model('out/models/mc1.xml.gz')
print(type(cobra_model))

# Loading an optlang model
optlang_model = load_model('out/problems/mc1.mps.7z',solver='gurobi') 
print(type(optlang_model))
# OR 
optlang_model = load_model('out/models/mc1.xml.gz')
print(type(optlang_model))

### Solving models

In [ ]:
from pymgpipe import solve_model, load_model

sol = solve_model(model='out/problems/mc1.mps.7z',solver='gurobi')

# OR 

m = load_model('out/problems/mc1.mps.7z', solver='gurobi')
target_reactions = get_reactions(m,regex='Diet_EX_.*')
sol = solve_model(model=m,reactions=target_reactions)

# OR 

sol = solve_model(model='out/models/mc1.xml.gz',solver='gurobi')

### Computin NMPCs

In [ ]:
from pymgpipe import compute_nmpcs

r = compute_nmpcs(samples='out/problems/',out_dir='out',diet_fecal_compartments=True,force=True)
r.nmpc

### Adding Diets

In [ ]:
from pymgpipe import *

cobra_model = load_cobra_model('out/models/mc1.xml.gz')
add_diet_to_model(cobra_model,'HighFatLowCarbDiet')
compute_nmpcs(cobra_model)